In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

### Importing data

In [3]:
# Path assignment
path = r'C:\Users\nodnarb\Documents\IC Basket Analysis'

In [4]:
# Import products.csv
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [5]:
# Import orders.csv
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

### Cleaning products.csv

In [7]:
# Looking for missing values in products.csv
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [8]:
# Identifying the missing values
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [9]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [10]:
# New dataframe without missing product names
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [11]:
# Finding duplicate rows
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [12]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [13]:
df_prods_clean.shape

(49677, 5)

In [14]:
# Deleting duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [15]:
df_prods_clean_no_dups.shape

(49672, 5)

In [16]:
# Exporting checked products df
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'))

### Cleaning orders.csv

### Step 1

In [19]:
# Consistency check
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
orders_day_of_week             0
order_time                     0
days_since_prior_order    206209
dtype: int64

### Step 2

In [21]:
# Interpretting values to find discrepancies
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


##### 'days_since_prior_order' has inconsistent count value, i.e all variables count 3.42xxxx while 'days_since_prior_order' count 3.21xxxx
This can imply missing values in this column; however missing values can mean that this was the first order by the customer and not a null value.

Another observation is that order numbers are capped at 100.


### Step 3

In [24]:
# Identifying mixed type data
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0:
        print (col)

In [25]:
df_ords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unnamed: 0              int64  
 1   order_id                int64  
 2   user_id                 int64  
 3   eval_set                object 
 4   order_number            int64  
 5   orders_day_of_week      int64  
 6   order_time              int64  
 7   days_since_prior_order  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 208.8+ MB


### Step 5

In [27]:
# Identifying missing values
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
orders_day_of_week             0
order_time                     0
days_since_prior_order    206209
dtype: int64

In [28]:
df_ords_nan = df_ords[df_ords['days_since_prior_order'].isnull()]

In [29]:
df_ords_nan

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_time,days_since_prior_order
0,0,2539329,1,prior,1,2,8,NaN
11,11,2168274,2,prior,1,2,11,NaN
26,26,1374495,3,prior,1,1,14,NaN
39,39,3343014,4,prior,1,6,11,NaN
45,45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...,...
3420930,3420930,969311,206205,prior,1,4,12,NaN
3420934,3420934,3189322,206206,prior,1,3,18,NaN
3421002,3421002,2166133,206207,prior,1,6,19,NaN
3421019,3421019,2227043,206208,prior,1,1,15,NaN


#### 'days_since_prior_order' has null values and all order numbers are 1 implying it is the first order placed therefore I will substitute all null values with 0

### Step 6

In [32]:
# Replacing all null values with 0 
df_ords['days_since_prior_order'].fillna(0, inplace=True)

C:\Users\nodnarb\AppData\Local\Temp\ipykernel_20268\3745439638.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ords['days_since_prior_order'].fillna(0, inplace=True)


In [33]:
df_ords.isnull().sum()

Unnamed: 0                0
order_id                  0
user_id                   0
eval_set                  0
order_number              0
orders_day_of_week        0
order_time                0
days_since_prior_order    0
dtype: int64

### Step 7

In [35]:
# Finding duplicated values
df_ords_dup = df_ords[df_ords.duplicated()]

In [36]:
df_ords_dup

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_time,days_since_prior_order


#### There are no duplicates

### Exporting

In [ ]:
# Exporting orders data checked
df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'))